#### Agent Executor with managing agent steps
In this example, we will be experimenting with the implementation of builing an agent executor that limits the number of chat messages that gets sent to the LLM during agent execution

In [1]:
import os
from dotenv import load_dotenv
import operator
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai.chat_models import ChatOpenAI
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain import hub
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, FunctionMessage
from langchain_core.agents import AgentAction, AgentFinish

In [3]:
load_dotenv()

True

In [4]:
tools = [TavilySearchResults(max_results=1), PythonREPLTool()]
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")
agent_runnable = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

In [5]:
from typing import TypedDict, Annotated, Sequence, Union, Tuple



class AgentState(TypedDict):
    input: Annotated[str, operator.setitem]
    chat_messages: Annotated[Sequence[BaseMessage], operator.add]
    agent_outcome: Annotated[Union[AgentAction, AgentFinish, None], operator.setitem]
    intermediate_steps: Annotated[Sequence[Tuple[AgentAction, str]], operator.add]

In [6]:
from langgraph.prebuilt import ToolExecutor


tool_executor = ToolExecutor(tools)

In [7]:
def run_agent(state: AgentState):
    inputs = state.copy()
    inputs["chat_messages"] = inputs["chat_messages"][-5:]
    inputs["intermediate_steps"] = inputs["chat_messages"][-5:]
    
    agent_outcome = agent_runnable.invoke(inputs)
    return {"agent_outcome": agent_outcome}

In [8]:
def execute_tools(state: AgentState):
    agent_action = state["agent_outcome"]
    result = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(result))]}

In [9]:
def should_continue(state: AgentState):
    if isinstance(state["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

In [11]:
from langgraph.graph import END, StateGraph


workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

workflow.add_edge("action", "agent")

workflow.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})

workflow.set_entry_point("agent")


app = workflow.compile()

In [12]:
inputs = {"input": "what is the weather in sf?", "intermediate_steps": [], "chat_messages": []}
for output in app.stream(inputs):
    print(list(output.values())[0])
    print("----")

{'agent_outcome': AgentFinish(return_values={'output': ''}, log='')}
----
{'input': 'what is the weather in sf?', 'chat_messages': [], 'agent_outcome': AgentFinish(return_values={'output': ''}, log=''), 'intermediate_steps': []}
----
